In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

plt.rcParams['figure.figsize'] = 20, 15
dic = {"size": 14}
matplotlib.rc('font', **dic)

# Predspracovanie kategorických stĺpcov

In [2]:
df = pd.read_csv("data/general_data.csv", index_col="EmployeeID")
df.drop(['EmployeeCount','StandardHours'],axis=1, inplace = True)
df = df[~df.isna().apply(any, axis=1)]

In [3]:
cat_cols = list(df.dtypes[df.dtypes == 'object'].index.values)
cat_cols.remove('Attrition')

In [4]:
df_cat = df[cat_cols]
df_cat.shape

(4382, 7)

# Label encoder
Zakódovanie hodnôt do čísel

In [5]:
df_cat.head()

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18
EmployeeID,,,,,,,
1,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married,Y
2,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single,Y
3,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married,Y
4,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married,Y
5,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single,Y


In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_cat.apply(encoder.fit_transform).head()

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18
EmployeeID,,,,,,,
1,2,2,1,0,0,1,0
2,1,1,1,0,6,2,0
3,1,1,4,1,7,1,0
4,0,1,1,1,1,1,0
5,2,1,3,1,7,2,0


PROBLEM týmto prístupom sme pridali lineárnu závislosť kde vlastne neexistuje.

# OneHot encoder
Pre každú jedinečnú hodnotu v stĺpci pridáme nový binárny stĺpec, ktorý bude mať hodnotu 1 len ak sa tam daná hodnota vyskytuje.

In [7]:
example = pd.DataFrame(['a', 'b', 'b', 'a', 'c'])
example

,0
0,a
1,b
2,b
3,a
4,c


In [8]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
result = encoder.fit_transform(example) # sparse matrix BORING
result

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [9]:
pd.DataFrame(result, columns=encoder.get_feature_names())

,x0_a,x0_b,x0_c
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [10]:
encoder = OneHotEncoder(sparse=False)
results = encoder.fit_transform(df_cat)
pd.DataFrame(results, columns=encoder.get_feature_names()) #.shape

,x0_Non-Travel,x0_Travel_Frequently,x0_Travel_Rarely,x1_Human Resources,x1_Research & Development,x1_Sales,x2_Human Resources,x2_Life Sciences,x2_Marketing,x2_Medical,...,x4_Manager,x4_Manufacturing Director,x4_Research Director,x4_Research Scientist,x4_Sales Executive,x4_Sales Representative,x5_Divorced,x5_Married,x5_Single,x6_Y
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4378,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4379,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4380,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [11]:
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform(df_cat)

array([[0., 0., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 0., 1., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.]])

# Predikcia

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, OneHotEncoder

In [13]:
df = pd.read_csv("data/general_data.csv", index_col="EmployeeID")
df.drop(['EmployeeCount','StandardHours'],axis=1, inplace = True) # Remove boring columns 
df = df[~df.isna().apply(any, axis=1)] 

cat_cols = list(df.dtypes[df.dtypes == 'object'].index.values)
cat_cols.remove('Attrition') 
num_cols = list(df.dtypes[df.dtypes != 'object'].index.values) + ["Attrition"]
num_cols.remove('Attrition')

In [14]:
scaler = MinMaxScaler()
polynomilas = PolynomialFeatures(3)
encoder = OneHotEncoder(sparse=False)

In [15]:
X = df[df.columns.difference(['Attrition'])]
y = df.Attrition 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
poly = polynomilas.fit_transform(X_train[num_cols])
scaled = scaler.fit_transform(poly)
encoded = encoder.fit_transform(X_train[cat_cols])
print(f"numerical shape is {scaled.shape}")
print(f"categorical  shape is {encoded.shape}")
X_train_proc = np.concatenate((scaled, encoded), axis=1)
X_train_proc.shape

numerical shape is (3505, 560)
categorical  shape is (3505, 27)


(3505, 587)

In [17]:
poly_test = polynomilas.transform(X_test[num_cols])
scaled_test = scaler.transform(poly_test)
encoded_test = encoder.transform(X_test[cat_cols])
print(f"numerical shape is {scaled_test.shape}")
print(f"categorical  shape is {encoded_test.shape}")
X_test_proc = np.concatenate((scaled_test, encoded_test), axis=1)
X_test_proc.shape

numerical shape is (877, 560)
categorical  shape is (877, 27)


(877, 587)

In [18]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_proc, y_train) 
y_pred = neigh.predict(X_test_proc)

print(f1_score(y_test, y_pred, average='micro'))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(classification_report(y_test,y_pred))

0.9224629418472063
0.8280610504774898
0.8759363804147215
              precision    recall  f1-score   support

          No       0.97      0.94      0.95       755
         Yes       0.69      0.81      0.74       122

    accuracy                           0.92       877
   macro avg       0.83      0.88      0.85       877
weighted avg       0.93      0.92      0.93       877

